In [1]:
""" Upload RDS table to BQ
-------------------------------------------------------------------------------

WARNING!!! DELETES DATASET!!!

Author: Rutger Hofste
Date: 20180712
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04
"""

# imports
import re
import os
import numpy as np
import pandas as pd
from retrying import retry
from datetime import timedelta
from sqlalchemy import *
pd.set_option('display.max_columns', 500)
from google.cloud import bigquery
import multiprocessing

SCRIPT_NAME = 'Y2018M07D16_RH_RDS_To_BQ_V01'
OUTPUT_VERSION = 5

TESTING = 0

DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
DATABASE_NAME = "database01"

INPUT_TABLE_NAME = "y2018m07d12_rh_ws_categorization_label_postgis_v01_v04"

OUTPUT_DATASET_NAME = "aqueduct30v01"
OUTPUT_TABLE_NAME = SCRIPT_NAME.lower() + "_v{:02.0f}".format(OUTPUT_VERSION)

print("Input Table: " , INPUT_TABLE_NAME, 
      "\nOutput Table: " , OUTPUT_DATASET_NAME,".",OUTPUT_TABLE_NAME)

Input Table:  y2018m07d12_rh_ws_categorization_label_postgis_v01_v04 
Output Table:  aqueduct30v01 . y2018m07d16_rh_rds_to_bq_v01_v05


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M07D16 UTC 19:35


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
F = open("/.password","r")
password = F.read().splitlines()[0]
F.close()

engine = create_engine("postgresql://rutgerhofste:{}@{}:5432/{}".format(password,DATABASE_ENDPOINT,DATABASE_NAME))


In [4]:
cpu_count = multiprocessing.cpu_count()
print("Power to the maxxx:", cpu_count)

Power to the maxxx: 16


In [5]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/.google.json"

In [6]:
bigquery_client = bigquery.Client()

In [7]:
dataset_id = 'my_new_dataset'
dataset_ref = bigquery_client.dataset(dataset_id)
bigquery_client.delete_dataset(dataset_ref)
dataset = bigquery.Dataset(dataset_ref)
dataset = bigquery_client.create_dataset(dataset)

In [8]:
sql = "SELECT DISTINCT pfafid_30spfaf06 FROM {} ORDER BY pfafid_30spfaf06".format(INPUT_TABLE_NAME)

In [9]:
df = pd.read_sql(sql,engine)

In [10]:
df.head()

,pfafid_30spfaf06
0,111011
1,111012
2,111013
3,111014
4,111015


In [11]:
df.shape

(16385, 1)

In [12]:
if TESTING:
    df = df[0:10]

In [13]:
df_split = np.array_split(df, cpu_count*100)

In [14]:
len(df_split)

1600

In [15]:
# pfaf_ids are been split in 1600 "packages" with appr. 11 rows each". 

In [16]:
df_split_first = df_split[0]
df_split_remainder = df_split[1:]

In [17]:
@retry(wait_exponential_multiplier=1000, wait_exponential_max=10000)
def df_to_bg(df):
    for index, row in df.iterrows():
        pfafid = row["pfafid_30spfaf06"]
        sql = "SELECT * FROM {} WHERE pfafid_30spfaf06 = {}".format(INPUT_TABLE_NAME,pfafid)
        df_basin = pd.read_sql(sql,engine)
        # add timestamp
        now = datetime.datetime.now()
        df_basin["processed_timestamp"] = pd.Timestamp(now)        
        df_basin.to_gbq(destination_table="{}.{}".format(OUTPUT_DATASET_NAME,OUTPUT_TABLE_NAME),
                        project_id = "aqueduct30",
                        if_exists= "append" )
        time.sleep(0.5)
        print(index)

In [18]:
# Upload first shard separately to initiate table schema
df_to_bg(df_split_first)

0
1
2
3
4
5
6
7
8
9
10


In [ ]:
p= multiprocessing.Pool()
results_buffered = p.map(df_to_bg,df_split_remainder)
p.close()
p.join()

2761
3861
1936
4136
1386
1111
3586
836
286
3036
2486
3311
2211
1661
11
561
2762
12
3861
4136
1111
562
2486
1661
2763
1936
1386
836
286
3311
3862
2211
3586
3036
13
1112
563
1662
2764
287
4136
14
2212
3037
2765
3587
836
1936
3861
3311
288
1661
564
2766
837
11
1937
3038
2211
3312
1386
3862
3586
289
3039
12
290
561
2761
1387
3587
2212
3313
3863
2762
836
1936
4136
562
1388
2213
3864
3036
11
2763
563
1937
837
2214
1661
3314
4137
286
1389
3586
3865
12
3037
3587
2215
1390
13
3866
561
3038
1391
562
14
1936
3311
3039
3867
1661
4136
836
2216
286
1392
563
15
3312
1937
3586
3040
1662
3868
4137
2487
1393
16
3313
3041
2764
1663
3869
3587
4138
2488
1938
1113
1394
2765
1664
3042
286
3870
561
2489
4139
11
3043
1939
1395
3871
2211
3311
562
287
2490
3044
3586
3872
3312
563
1661
288
2491
3045
837
3587
289
4136
2492
11
838
1386
2211
564
3872
3046
2493
1936
12
1387
2212
3311
839
2761
1661
3873
3586
3047
565
840
286
2762
3587
1662
4136
1114
3048
3874
11
2763
1386
3311
1936
1663
4137
2211
1115
3875
1387
3312
8

In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)